In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.optim as optim
from twa.utils import ensure_dir, write_yaml, read_yaml, glob_re
from twa.data import FlowSystemODE, topo_point_vs_cycle
from twa.evaluate import Evaluator
from sklearn.decomposition import PCA
import random
random.seed(0)
torch.manual_seed(0)

In [ ]:
# from sklearn.metrics import mutual_info_score
# from sklearn.feature_selection import mutual_info_regression

# x = np.arange(0, 1, 0.1).reshape((-1, 1))
# y = np.random.permutation(x)
# # print(x)
# # print(y)
# # print(mutual_info_score(x, y))
# print(mutual_info_regression(x, y)[0])

# Results

In [ ]:
outdir='../output/'

# read all repeated runs of these experiments
exp_descs = {} 
exp_descs['simple_oscillator_nsfcl_angle_atten_AttentionwFC'] = 'Our Model'

# exp_descs['dropout5'] = 'dropout = 0.5'
# exp_descs['kernel5'] = 'kernel size = 5'
# exp_descs['latent5'] = 'latent dimension = 5'
# exp_descs['latent20'] = 'latent dimension = 20'
# exp_descs['channels32'] = 'conv channels = 32'
# exp_descs['conv3'] = 'conv layers = 3'
exp_descs['p2v_linear'] = 'Phase2Vec Latent Representation'
exp_descs['vfpoly_linear'] = 'Vector Field Learned Representation'
exp_descs['params_linear'] = 'Fitted Parameter Coefficients'

# exp_descs['simple_oscillator_nsfcl_angle_atten_AttentionwFC_lr3'] = 'Our Model lr3'
# exp_descs['p2v_linear_lr3'] = 'Phase2Vec Latent Representation lr3'
# exp_descs['vfpoly_linear_lr3'] = 'Vector Field Learned Representation lr3'
# exp_descs['params_linear_lr3'] = 'Fitted Parameter Coefficients lr3'

# exp_descs['simple_oscillator_nsfcl_angle_atten_AttentionwFC_lr5'] = 'Our Model lr5'
# exp_descs['p2v_linear_lr5'] = 'Phase2Vec Latent Representation lr5'
# exp_descs['vfpoly_linear_lr5'] = 'Vector Field Learned Representation lr5'
# exp_descs['params_linear_lr5'] = 'Fitted Parameter Coefficients lr5'

# exp_descs['simple_oscillator_nsfcl_angle_atten_AttentionwFC_ne10'] = 'Our Model ne10'
# exp_descs['p2v_linear_ne10'] = 'Phase2Vec Latent Representation ne10'
# exp_descs['vfpoly_linear_ne10'] = 'Vector Field Learned Representation ne10'
# exp_descs['params_linear_ne10'] = 'Fitted Parameter Coefficients ne10'

# exp_descs['simple_oscillator_nsfcl_angle_atten_AttentionwFC_ne40'] = 'Our Model ne40'
# exp_descs['p2v_linear_ne40'] = 'Phase2Vec Latent Representation ne40'
# exp_descs['vfpoly_linear_ne40'] = 'Vector Field Learned Representation ne40'
# exp_descs['params_linear_ne40'] = 'Fitted Parameter Coefficients ne40'




sys_descs = {}
sys_descs['simple_oscillator_noaug'] = 'SO'
sys_descs['simple_oscillator_nsfcl'] = 'Augmented SO'
sys_descs['suphopf'] = 'Supercritical Hopf'
# sys_descs['subhopf'] = 'Subcritical Hopf'
sys_descs['lienard_poly'] = 'Lienard Poly'
sys_descs['lienard_sigmoid'] = 'Lienard Sigmoid'
sys_descs['vanderpol'] = 'Van der Pol'
sys_descs['bzreaction'] = 'BZ Reaction'
sys_descs['selkov2'] = 'Sel\'kov'

selected_syss = list(sys_descs.keys()) # classical synthetic systems
print(selected_syss)

sys_descs['repressilator'] = 'Repressilator'
sys_descs['pancreas_clusters_random_bin'] = 'Pancreas'


ode_syss = {k: k for k in sys_descs.keys()}
ode_syss['simple_oscillator_noaug'] = 'simple_oscillator'
ode_syss['simple_oscillator_nsfcl'] = 'simple_oscillator'
ode_syss['pancreas_clusters_random_bin'] = None
ode_syss['selkov2'] = 'selkov'


# # NOISED
# sys_descs = {}
# sys_descs['simple_oscillator_noaug_noise0.1'] = 'SO'
# sys_descs['simple_oscillator_nsfcl_noise0.1'] = 'Augmented SO'
# sys_descs['suphopf_noise0.1'] = 'Supercritical Hopf'
# # sys_descs['subhopf_noise0.1'] = 'Subcritical Hopf'
# sys_descs['lienard_poly_noise0.1'] = 'Lienard Poly'
# sys_descs['lienard_sigmoid_noise0.1'] = 'Lienard Sigmoid'
# sys_descs['vanderpol_noise0.1'] = 'Van der Pol'
# sys_descs['bzreaction_noise0.1'] = 'BZ Reaction'
# sys_descs['selkov2_noise0.1'] = 'Sel\'kov'

# selected_syss = list(sys_descs.keys()) # classical synthetic systems
# print(selected_syss)

# sys_descs['repressilator'] = 'Repressilator'
# sys_descs['pancreas_clusters_random_bin'] = 'Pancreas'


# ode_syss = {k: k for k in sys_descs.keys()}
# ode_syss['pancreas_clusters_random_bin'] = None
# ode_syss['simple_oscillator_noaug_noise0.1'] = 'simple_oscillator'
# ode_syss['simple_oscillator_nsfcl_noise0.1'] = 'simple_oscillator'
# ode_syss['suphopf_noise0.1'] = 'suphopf'

# ode_syss['lienard_poly_noise0.1'] = 'lienard_poly'
# ode_syss['lienard_sigmoid_noise0.1'] = 'lienard_sigmoid'
# ode_syss['vanderpol_noise0.1'] = 'vanderpol'
# ode_syss['bzreaction_noise0.1'] = 'bzreaction'
# ode_syss['selkov2_noise0.1'] = 'selkov'





EV = Evaluator(exp_descs=exp_descs, sys_descs=sys_descs, ode_syss=ode_syss, verbose=True, outdir=outdir)

exp = 'simple_oscillator_nsfcl_angle_atten_AttentionwFC'

In [ ]:
# average accuracies across repeats
res = EV.get_stats(syss=['pancreas_clusters_random_bin'], stats=['Accu'])
print(res)
EV.print_stat(syss=['pancreas_clusters_random_bin']) # for latex table

In [ ]:
# average accuracies across repeats
res = EV.get_stats(syss=selected_syss, stats=['Accu'])
print(res)
EV.print_stat(syss=selected_syss) # for latex table

In [ ]:
# accuracy, sensitivity and specificity across repeats
res = EV.get_stats(syss=selected_syss, stats=['Accu', 'Sens', 'Spec'])
print(res)

In [ ]:
# # print noise evaluation
# exp = 'p2v_linear'
# exp = 'vfpoly_linear'
exp = 'params_linear'
df = EV.gather_stats_noised(exp=exp, desc='noise')
df
# df = EV.gather_stats_noised(exp=exp, desc='masked')

In [ ]:
# # for subhopf, get confidence? on each regime
# sys = 'subhopf'
# sysp = EV.sysps[sys]
# vals = np.zeros((len(sysp),2))
# i = 0
# for exp_desc, outputs in EV.exp_outputs.items():
#     if sys not in outputs: 
#         # print('Missing')
#         continue
#     i += 1
#     vals += outputs[sys] # (outputs[sys] > 0)
# vals /= i

In [ ]:
# print(vals[sysp[:,0] < -0.25, 0].mean())
# print(vals[(sysp[:,0] > -0.25) & (sysp[:,0] < 0), 0].mean())
# print(vals[(sysp[:,0] > 0), 0].mean())

# print(vals[sysp[:,0] < -0.25, 1].mean())
# print(vals[(sysp[:,0] > -0.25) & (sysp[:,0] < 0), 1].mean())
# print(vals[(sysp[:,0] > 0), 1].mean())

In [ ]:
# plotting confidence and resulting bifurcation curves
sselected_syss = [s for s in selected_syss if s != 'vanderpol'] # exclude vanderpol because of single parameter
ref_sys = 'simple_oscillator_nsfcl'
# EV.plot_bifurcation(exp, syss=['subhopf'], verbose=True)
exp = 'p2v_linear'
exp = 'vfpoly_linear'
exp = 'params_linear'
# exp = 'simple_oscillator_nsfcl_angle_atten_AttentionwFC'

EV.plot_bifurcation(exp, syss=sselected_syss, verbose=True)

In [ ]:
# sys = 'suphopf'
from sklearn.feature_selection import mutual_info_regression
from twa.utils import plot_tradeoff
from scipy.stats import spearmanr

sselected_syss = [s for s in selected_syss if s != 'vanderpol'] # exclude vanderpol because of single parameter
ncols = len(sselected_syss)
fig, ax = plt.subplots(1,ncols,figsize=(5*ncols,7), sharey=True, constrained_layout=True)
bins = 10
xlabel=''
ylabel=''
for isys, sys in enumerate(sselected_syss):
    sysp = EV.sysps[sys]
    bifurp = EV.bifurps[sys]
    fit_value = np.mean([np.sign(EV.exp_outputs[exp_rep][sys]) for exp_rep in EV.exp_repeats[exp] if sys in EV.exp_outputs[exp_rep].keys()], axis=0)
    df_cyc = pd.DataFrame({ 'dist': np.abs(bifurp[bifurp > 0]), 'confidence': np.abs(fit_value[bifurp > 0,0])})
    df_pt = pd.DataFrame({ 'dist': np.abs(bifurp[bifurp < 0]), 'confidence': np.abs(fit_value[bifurp < 0,0])})
    
    # corr_pt = mutual_info_regression(df_pt[['dist']], df_pt[['confidence']])[0]
    # corr_cyc = mutual_info_regression(df_cyc[['dist']], df_cyc[['confidence']])[0]
    
    # corr_pt = np.corrcoef(df_pt['dist'], df_pt['confidence'])[0,1]
    # corr_cyc = np.corrcoef(df_cyc['dist'], df_cyc['confidence'])[0,1]
    
    corr_pt = spearmanr(df_pt['dist'], df_pt['confidence']).correlation
    corr_cyc = spearmanr(df_cyc['dist'], df_cyc['confidence']).correlation

    print(sys, corr_pt, corr_cyc)
    tit = f'{sys_descs[sys]}\n({corr_pt:.2f}, {corr_cyc:.2f})'
    # tit = f'({corr_pt:.2f}, {corr_cyc:.2f})'

    
    df_cyc['dist_bin'] = pd.cut(df_cyc['dist'], bins=bins)
    df_cyc['dist_mid'] = df_cyc['dist_bin'].apply(lambda x: x.mid).astype(float)
    plot_tradeoff(df_cyc, xcol='dist_mid', ycol='confidence', groupby='dist_mid', plot_std=1, color_mean='r', color_std='r', xlabel=xlabel, ylabel=ylabel, ax=ax[isys])

    df_pt['dist_bin'] = pd.cut(df_pt['dist'], bins=bins)
    df_pt['dist_mid'] = df_pt['dist_bin'].apply(lambda x: x.mid).astype(float)
    plot_tradeoff(df_pt, xcol='dist_mid', ycol='confidence', groupby='dist_mid', plot_std=1, color_mean='b', color_std='b', xlabel=xlabel, ylabel=ylabel, ax=ax[isys], title=tit)

    # ax[isys].set_title(tit)
    # ax[isys].scatter(df_cyc['dist'], df_cyc['confidence'], c='r', alpha=0.5)
    # ax[isys].scatter(df_pt['dist'], df_pt['confidence'], c='b', alpha=0.5)

ax[0].set_ylabel('Confidence', fontsize=30)
fig.supxlabel('Distance from bifurcation', fontsize=30)
# fig.supylabel('Confidence', fontsize=25)

In [ ]:
bins = 20


In [ ]:
import seaborn as sns
# plt.scatter(sysp[:,0], fit_value[:,1])
df = pd.DataFrame({r'$\mu$': sysp[:,0], 'probability predicted cycle': fit_value[:,1]})
df['topo'] = 'Point'
df.loc[df[r'$\mu$'] > 0, 'topo'] = 'Periodic'
sns.boxplot(
    df, x='topo', y='probability predicted cycle', palette="vlag", order=['Point', 'Periodic', ], # hue="method",
    #whis=[0, 100], width=.6, palette="vlag"
)

In [ ]:

fit_value = {}
for sys in selected_syss:
    fit_value[sys] = np.mean([np.sign(EV.exp_outputs[exp_rep][sys]) for exp_rep in EV.exp_repeats[exp] if sys in EV.exp_outputs[exp_rep].keys()], axis=0)
    
        # fit_value = {sys: np.mean([np.sign(EV.exp_outputs[exp_rep][sys]) , axis=0) for sys in syss if sys in EV.exp_outputs[exp_rep].keys()}
        # fit_value = {sys: np.mean([np.sign(EV.exp_outputs[exp_rep][sys]) for exp_rep in EV.exp_repeats[exp]], axis=0) for sys in syss}

cyc_attr_idx = 1
for isys, sys in enumerate(selected_syss):
    sys = 'suphopf'
    bifurp = EV.bifurps[sys]
    y_cyc = fit_value[sys][:, cyc_attr_idx]
    corr = np.corrcoef(np.abs(bifurp), np.abs(y_cyc))[0,1]
    print(sys, corr)
    break


In [ ]:
# plotting confidence and resulting bifurcation curves

EV.plot_bifurcation(exp, syss=['repressilator'])

In [ ]:
# plot pancreas predicted cycle scores
EV.plot_pancreas(exp)

In [ ]:
exp_descs2 = {}
exp_descs2['simple_oscillator_nsfcl_angle_atten_AttentionwFC'] = 'Our Model'
exp_descs2['simple_oscillator_nsfcl_angle_noatten_AttentionwFC'] = 'No Attention'
exp_descs2['simple_oscillator_nsfcl_vector_atten_AttentionwFC'] = 'From Vectors'
exp_descs2['simple_oscillator_noaug_angle_atten_AttentionwFC'] = 'No Augmentation'

exp_descs2['simple_oscillator_noaug_vector_noatten_AttentionwFC'] = 'CNN-baseline'


EV2 = Evaluator(exp_descs=exp_descs2, sys_descs=sys_descs, ode_syss=ode_syss, outdir=outdir)


In [ ]:
res = EV2.get_stats(syss=selected_syss, stats=['Accu'])
print(res)
EV2.print_stat(syss=selected_syss) # for latex table

In [ ]:
# EV2.plot_output_distribution()